<a href="https://colab.research.google.com/drive/your-notebook-name" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tune BioBERT on PubMed Abstracts

This notebook fine-tunes BioBERT (`allenai/biobert-v1.1-pubmed`) on PubMed abstracts using Masked Language Modeling (MLM) for biomedical NLP tasks, such as improving spellchecking in medical texts (e.g., correcting 'arbitysratsddion' to 'arteries'). Abstracts are downloaded from NCBI's PubMed baseline FTP server, preprocessed, and used for training in Google Colab with GPU support. The model is saved to Google Drive for local use in a spellchecker project.

**Setup**: Google Colab (GPU), PyTorch, NCBI PubMed abstracts.
**Output**: Fine-tuned BioBERT model and tokenizer saved to `/content/drive/MyDrive/biobert_finetuned`.

In [1]:
!pip install transformers==4.28.0 datasets==2.10.0 torch==1.13.1 sentencepiece==0.1.97 numpy==1.24.2 lxml==4.9.2

# Verify installation
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch version: 1.13.1+cu117
CUDA available: True
GPU: Tesla T4


In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Set output directory
output_dir = '/content/drive/MyDrive/biobert_finetuned'
import os
os.makedirs(output_dir, exist_ok=True)

Mounted at /content/drive


In [3]:
import urllib.request
import gzip
import xml.etree.ElementTree as ET
import re
from nltk.tokenize import TreebankWordTokenizer

# Download a single PubMed baseline file
url = 'https://ftp.ncbi.nlm.nih.gov/pubmed/baseline/pubmed25n0001.xml.gz'
local_file = '/content/pubmed25n0001.xml.gz'

try:
    print(f"Downloading {url}...")
    urllib.request.urlretrieve(url, local_file)
except Exception as e:
    print(f"Error downloading file: {e}")
    raise

# Extract and preprocess abstracts
def extract_abstracts(xml_gz_file):
    abstracts = []
    tokenizer = TreebankWordTokenizer()
    with gzip.open(xml_gz_file, 'rt', encoding='utf-8') as f:
        tree = ET.parse(f)
        root = tree.getroot()
        for article in root.findall('.//Article'):
            abstract = article.find('.//Abstract/AbstractText')
            if abstract is not None and abstract.text:
                text = abstract.text.lower()
                text = re.sub(r'[\r\n]+', ' ', text)
                text = re.sub(r'[^\x00-\x7F]+', ' ', text)
                tokenized = tokenizer.tokenize(text)
                text = ' '.join(tokenized)
                text = re.sub(r"\s's\b", "'s", text)
                abstracts.append(text)
    return abstracts

abstracts = extract_abstracts(local_file)
print(f"Extracted {len(abstracts)} abstracts")

# Save abstracts for inspection
with open('/content/pubmed_abstracts.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(abstracts))

# Create a dataset
from datasets import Dataset
dataset = Dataset.from_dict({'text': abstracts})
print(dataset)
print(dataset[0]['text'] if len(dataset) > 0 else 'No abstracts')

Extracted 15377 abstracts
Dataset({
    features: ['text'],
    num_rows: 15377
})
( -- ) -alpha-bisabolol has a primary antipeptic action depending on dosage , which is not caused by an alteration of the ph-value. the proteolytic activity of pepsin is reduced by 50 percent through addition of bisabolol in the ratio of 1/0.5. the antipeptic action of bisabolol only occurs in case of direct contact. in case of a previous contact with the substrate , the inhibiting effect is lost .


In [4]:
from transformers import BertTokenizer, DataCollatorForLanguageModeling

# Load BioBERT tokenizer
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')

# Preprocess function
def preprocess_function(examples):
    encodings = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    return encodings

# Apply preprocessing
encoded_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text'])

# Data collator for MLM
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

AttributeError: module 'numpy' has no attribute 'dtypes'

In [5]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 1.13.1+cu117
CUDA available: True


In [ ]:
from transformers import BertForMaskedLM, Trainer, TrainingArguments

# Load BioBERT model
model = BertForMaskedLM.from_pretrained('dmis-lab/biobert-base-cased-v1.1')

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=f'{output_dir}/logs',
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
    fp16=True if torch.cuda.is_available() else False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    data_collator=data_collator,
)

# Train
trainer.train()

# Save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f'Model saved to {output_dir}')

In [ ]:
from transformers import pipeline

# Test the fine-tuned model
fill_mask = pipeline('fill-mask', model=output_dir, tokenizer=output_dir)
test_sentence = 'Hypertension is a [MASK] condition.'
results = fill_mask(test_sentence)
for result in results:
    print(f"Token: {result['token_str']}, Score: {result['score']:.4f}")

# Test spellchecker-relevant sentence
test_spell = 'The patient has [MASK] blood pressure.'
results_spell = fill_mask(test_spell)
for result in results_spell:
    print(f"Token: {result['token_str']}, Score: {result['score']:.4f}")

In [ ]:
# Zip the model for download
!zip -r /content/biobert_finetuned.zip /content/drive/MyDrive/biobert_finetuned

from google.colab import files
files.download('/content/biobert_finetuned.zip')
print('Download the zip file to your local machine')